In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyper-parameters
num_epochs = 4
batch_size = 4
learning_rate = .001

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))])

train_dataset = torchvision.datasets.CIFAR10(root = './data', train = True, download= True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root = './data', train = False, download= True, transform=transform)

                                             
train_loader = DataLoader(dataset = train_dataset, shuffle= True, batch_size = batch_size )
test_loader = DataLoader(dataset = test_dataset, shuffle= False, batch_size = batch_size )


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        
        x = x.view(-1, 16*5*5)#
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        # softmax layer
        return x
    
    
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i,(images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        output = model(images)
        
        loss = criterion(output, labels)
        loss.backward()
        
        optimizer.step()
        
        optimizer.zero_grad()
        
        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            
            
            
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        #print(f'Accuracy of {classes[i]}: {acc} %')
        

Files already downloaded and verified
Files already downloaded and verified


/home/avinash/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448216815/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch [1/4], Step [2000/12500], Loss: 2.2847
Epoch [1/4], Step [4000/12500], Loss: 2.3068
Epoch [1/4], Step [6000/12500], Loss: 2.2895
Epoch [1/4], Step [8000/12500], Loss: 2.2824
Epoch [1/4], Step [10000/12500], Loss: 2.2675
Epoch [1/4], Step [12000/12500], Loss: 2.3709
Epoch [2/4], Step [2000/12500], Loss: 1.8394
Epoch [2/4], Step [4000/12500], Loss: 2.0966
Epoch [2/4], Step [6000/12500], Loss: 1.4562
Epoch [2/4], Step [8000/12500], Loss: 1.9803
Epoch [2/4], Step [10000/12500], Loss: 2.2259
Epoch [2/4], Step [12000/12500], Loss: 1.9156
Epoch [3/4], Step [2000/12500], Loss: 1.4866
Epoch [3/4], Step [4000/12500], Loss: 1.2400
Epoch [3/4], Step [6000/12500], Loss: 1.7309
Epoch [3/4], Step [8000/12500], Loss: 1.5280
Epoch [3/4], Step [10000/12500], Loss: 1.1756
Epoch [3/4], Step [12000/12500], Loss: 2.0063
Epoch [4/4], Step [2000/12500], Loss: 1.2196
Epoch [4/4], Step [4000/12500], Loss: 1.7118
Epoch [4/4], Step [6000/12500], Loss: 1.5583
Epoch [4/4], Step [8000/12500], Loss: 2.0298
Epoc